In [26]:
import numpy as np
import pandas as pd
import itertools

import copy
import datetime

from scipy.optimize import fsolve
from scipy.special import gamma


In [27]:
class City(object):
    
    def __init__(self, width, height, hospital_count, subregion_count):
        
        self.width = width
        self.height = height
        #for simplicity, subregion_count = base_count
        
        hospital_x = np.linspace(0, self.width, num = hospital_count + 2)
        self.hospital_location = [(hospital_x[i+1], self.height/2) for i in range(hospital_count)]
        
        self.subregion_count = subregion_count + (subregion_count % 2)
        subregion_x = np.linspace(0, self.width, num = int(self.subregion_count/2 + 2))
        subregion_y = np.linspace(0, self.height, num = 4)
        base_location = [[(subregion_x[i+1], subregion_y[j+1]) for j in range(2)] for i in range(int(self.subregion_count/2))]
        self.base_location = list(itertools.chain(*base_location))

In [28]:
def gen_call_arrival_loc(city):
    return (np.random.uniform(0, city.width),np.random.uniform(0, city.height))

In [29]:
def gen_call_arrival_time(mean = 4):
    return np.random.exponential(mean)

In [30]:
def gen_call_priority(priority_rate = 0.25):
    return int(np.random.uniform(0,1) < priority_rate)

In [31]:
def gen_atlocation_service_time():
    return np.random.exponential(12)

In [32]:
def weibull_param_relationship(scale, mean, stddev):
    return stddev**2/mean**2 - gamma(1 + 2/scale)/(gamma(1+1/scale)**2) + 1

def get_weibull_parameters(mean = 30, stddev = 13):
    scale = fsolve(weibull_param_relationship, 1,args=(mean, stddev))
    shape = mean / gamma(1+1/scale)
    
    return scale, shape

def gen_athospital_service_time():
    scale, shape = get_weibull_parameters(mean = 30, stddev = 13)
    return (scale * np.random.weibull(shape))[0]

In [33]:
class Call(object):
    
    def __init__(self, call_index, location, time_arrival, interarrival_time, priority = 0):
        
        self.call_index = call_index
        self.location = location
        
        #initialize the call to be unassigned
        #-1-unassigned, 0 to (N-1) assigned ambulance index 
        self.status = -1
        self.arrival_time = time_arrival
        self.next_arrival_time = self.arrival_time + interarrival_time
        self.priority = priority

In [34]:
def get_distance(location1, location2):
    
    distance = np.abs(location1[0] - location2[0]) + np.abs(location1[1] - location2[1])
    return distance

In [35]:
def get_ambulance_travel_time(distance, speed = 30):
    return distance/speed

In [36]:
class Ambulance(object):
    
    def __init__(self, base, speed = 30):
        
        #fix the ambulance's home base
        self.base = base
        
        #set the ambulance's travel speed
        self.speed = speed
        
        #initialize the ambulance idle at home base
        
        #status code: 0-idle at base, 1-going to scene of call, 2-serving at scene of call, 3-going to hospital
        #4-transferring patient at hospital, 5-returning to base
        self.status = 0
        self.origin = base
        self.destination = base # if destination = origin, ambulance is stationary
        self.time = 0
        self.endtime = np.inf
        self.call = (-1, -1) # -1 if not assigned to any call, second one indicates priority
        
        
    def update_status(self, status, origin, destination, time, endtime):
        self.status = status
        
        self.origin = origin
        self.destination = destination
        self.time = time
        self.endtime = endtime
        
    def redployment(self, base = None):
        
        if base is None:
            base = self.base
        
        #current heuristic: return to home base
        distance = get_distance(self.origin, base)
        travel_time = get_ambulance_travel_time(distance, self.speed)
        self.update_status(5, self.origin, self.base, copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        self.call = (-1, -1)
        
    def return_to_base(self):
        self.update_status(0, self.base, self.base, copy.deepcopy(self.endtime), np.inf)
        
        
    def assign_to_call(self, call, time, index):
        
        distance = get_distance(self.origin, call.location)
        travel_time = get_ambulance_travel_time(distance, self.speed)
        self.update_status(1, self.origin, call.location, time, time + travel_time)
        self.call = (call.call_index, call.priority) # updated assigned call index
        
        call.status = index
        
        
    def reach_call_location(self, call, callList):
    
        atlocation_servicetime = gen_atlocation_service_time()
        self.update_status(2, call.location, call.location, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + atlocation_servicetime)

        callList.pop(call.call_index)
    
    def transport_to_hospital(self, city):
        
        #transport to nearest hospital
        
        hospital_list = city.hospital_location
        
        min_distance = np.inf
        nearest_hospital = hospital_list[0]
        for hospital in hospital_list:
            distance = get_distance(self.origin, hospital)
            if distance < min_distance:
                min_distance = distance
                nearest_hospital = hospital
        
        travel_time = get_ambulance_travel_time(min_distance, self.speed)
        self.update_status(3, self.origin, hospital, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        
    def reach_hospital(self):
        hospital_servicetime = gen_athospital_service_time()
        self.update_status(4, copy.deepcopy(self.destination), copy.deepcopy(self.destination), 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + hospital_servicetime)




In [37]:
def arrival(call_index, ambulanceList, callList, time_arrival, M, timeline, 
            call_mean = 4, priority_rate = 0.25):
    
    call = Call(call_index, gen_call_arrival_loc(city), time_arrival, 
                gen_call_arrival_time(call_mean), priority = gen_call_priority(priority_rate))
    
    i = timeline.shape[0]
    timeline.loc[i] = [call_index, call.priority, '', 0, time_arrival]
    if len(callList) >= M:
        # print("New call arrived. No more capacity. Reject call.")
        i = timeline.shape[0]
        timeline.loc[i] = [call_index, call.priority, '', 7, time_arrival]
    
    else:
    
        # print("New call arrived. Add to call list.")
        callList[call_index] = call

        assign = -1
        index = 0

        nearest_distance = np.inf
        for ambulance in ambulanceList:

            if ambulance.status == 0:
                distance = get_distance(ambulance.origin, call.location)
                if distance < nearest_distance:
                    nearest_distance = distance
                    assign = index

            index = index + 1

        if assign > -1:
            # when the call arrives, there are ambulances idle at base, so assign the call to the nearest ambulance
            # print("Idle ambulance at base available. Assign. Now travelling to call location.")
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, call.priority, assign, 1, time_arrival]
            ambulanceList[assign].assign_to_call(call, time_arrival, assign)

        # else:
            # print("No available ambulance at the moment.")
            
    time_arrival = call.next_arrival_time
    
    return ambulanceList, callList, time_arrival, timeline

In [38]:
def get_first(ambulance, callList, M, k):
    
    call_index = -1
    min_arrival_time = np.inf
    for call_id, call in callList.items():
        if call.status == -1:
            call_time = call.arrival_time
            if call_time < min_arrival_time:
                call_index = call_id
                min_arrival_time = call_time
        
    return call_index


In [39]:
def get_first_highpriority(ambulance, callList, M, k):
    
    # Assign based on FCFS within priority
    # assume only high and low, two levels or priority
    
    
    high_call_index = -1
    low_call_index = -1
    
    high_min_arrival_time = np.inf
    low_min_arrival_time = np.inf
    for call_id, call in callList.items():
        if call.status == -1:
            call_time = call.arrival_time
            
            if call.priority == 1:
                if call_time < high_min_arrival_time:
                    high_call_index = call_id
                    high_min_arrival_time = call_time
            
            else:
                if call_time < low_min_arrival_time:
                    low_call_index = call_id
                    low_min_arrival_time = call_time
            

    if high_call_index > -1:
        return high_call_index
    else:
        return low_call_index
    

In [40]:
def get_nearest(ambulance, callList, M, k):
    
    ambulance_loc = ambulance.origin
    
    min_distance = np.inf
    call_index = -1
    
    for call_id, call in callList.items():
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            if distance < min_distance:
                call_index = call_id
                min_distance = distance
    
    if call_index > -1:
        # there is some unassigned call in queue
        return call_index
    else:
        # all calls in queue have been assigned
        return -1

In [41]:
def get_nearest_threshold(ambulance, callList, M, k):
    # only serve unassigned calls within k distance from the ambulance
    
    ambulance_loc = ambulance.origin
    
    min_distance = k
    call_index = -1
    
    for call_id, call in callList.items():
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            if distance <= min_distance:
                call_index = call_id
                min_distance = distance
    
    if call_index > -1:
        # there is some unassigned call in queue
        return call_index
    else:
        # all calls in queue have been assigned
        return -1

In [42]:
def get_nearest_threshold_else_fcfs(ambulance, callList, M, k):
    # serve unassigned calls within k distance from the ambulance
    # if no unassigned calls within the threshold, perform according to fcfs
    
    ambulance_loc = ambulance.origin
    
    min_distance = k
    call_index = -1
    min_arrival_time = np.inf
    
    for call_id, call in callList.items():
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            if distance < min_distance:
                call_index = call_id
                min_distance = distance
            elif distance > k:
                if call.arrival_time < min_arrival_time:
                    call_index = call_id
                    min_arrival_time = call.arrival_time
    
    if call_index > -1:
        # there is some unassigned call in queue
        return call_index
    else:
        # all calls in queue have been assigned
        return -1

In [43]:
def get_fcfs_threshold_else_nearest(ambulance, callList, M, k):
    # serve fcfs within a threshold, else serve nearest
    
    ambulance_loc = ambulance.origin
    
    min_distance = np.inf
    nearest_call_index = -1
    min_arrival_time = np.inf
    fcfs_call_index = -1
    
    for call_id, call in callList.items():
        
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            
            if distance < k:
                if call.arrival_time < min_arrival_time:
                    fcfs_call_index = call_id
                    min_arrival_time = call.arrival_time
                    
            else:
                
                if distance < min_distance:
                    nearest_call_index = call_id
                    min_distance = distance
    
    if fcfs_call_index > -1:
        return fcfs_call_index
    else:
        return nearest_call_index

In [44]:
def get_nearest_highpriority(ambulance, callList, M, k):
    
    ambulance_loc = ambulance.origin
    
    min_high_distance = np.inf
    min_low_distance = np.inf
    high_call_index = -1
    low_call_index = -1
    
    for call_id, call in callList.items():
        if call.status == -1:
            distance = get_distance(call.location, ambulance_loc)
            
            if call.priority == 1:
                if distance < min_high_distance:
                    high_call_index = call_id
                    min_high_distance = distance
            else:
                if distance < min_low_distance:
                    low_call_index = call_id
                    min_low_distance = distance
    
    if high_call_index > -1:
        return high_call_index
    elif low_call_index > -1:
        return low_call_index
    else:
        return -1

In [45]:
def get_next_event(policy, time_arrival, ambulanceList, callList, timeline, 
                   city, M, time_threshold, 
                   distance_threshold = 2, call_mean = 4, call_priority = 0.25):
    
    ambulanceEndTime_min = np.inf
    index_min = -1
    index = 0
    for ambulance in ambulanceList:
        if ambulance.endtime < ambulanceEndTime_min:
            ambulanceEndTime_min = copy.deepcopy(ambulance.endtime)
            index_min = index
        
        index = index + 1
    
    next_event_time = min(time_arrival, ambulanceEndTime_min)
    
    if next_event_time == time_arrival:
        # print("New call arrived.")
        all_call = set(timeline['Call'])
        call_index = len(all_call) if -1 not in all_call else len(all_call)-1
        ambulanceList, callList, time_arrival, timeline = \
        arrival(call_index, ambulanceList, callList, time_arrival, M, timeline, call_mean, call_priority)
        
    else:
        if ambulanceList[index_min].status == 1:
            # print("Now reach call location. Start at-location treatment. Remove call from call list.")
            call_index, priority = ambulanceList[index_min].call
            call = callList[call_index]
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, priority, index_min, 2, next_event_time]
            ambulanceList[index_min].reach_call_location(call, callList)
            
        elif ambulanceList[index_min].status == 2:
            # print("Now finish at-location treatment. Start going to hospital.")
            call_index, priority = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, priority, index_min, 3, next_event_time]
            ambulanceList[index_min].transport_to_hospital(city)
            
        elif ambulanceList[index_min].status == 3:
            # print("Now reach hospital. Start transferring patient to hospital.")
            call_index, priority = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, priority, index_min, 4, next_event_time]
            ambulanceList[index_min].reach_hospital()
            
        elif ambulanceList[index_min].status == 4:
            
            # print("Now finish transfering patient to hospital. Decide next step (assign to call or return to base).")
            call_index, priority = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, priority, index_min, 5, next_event_time]
            
            if len(callList) == 0:
                # print("Return to base.")
                ambulanceList[index_min].redployment()
            else:
                # print("Call waiting. Assign to call in queue according to policy.")
                call_index = policy(ambulanceList[index_min], callList, M, 
                                    time_threshold * ambulanceList[index_min].speed/distance_threshold)
                
                if call_index == -1:
                    # calls in callList have all been assigned with an ambulance, or exceed distance threshold
                    ambulanceList[index_min].redployment()
                    
                else:
                    i = timeline.shape[0]
                    call = callList[call_index]
                    timeline.loc[i] = [call_index, call.priority, index_min, 1, next_event_time]
                    ambulanceList[index_min].assign_to_call(call, next_event_time, index_min)
                    
        elif ambulanceList[index_min].status == 5:
            i = timeline.shape[0]
            timeline.loc[i] = [-1, -1, index_min, 6, next_event_time]
            # print("Now reployed ambulance reach base. Start idling.")
            ambulanceList[index_min].return_to_base()
    
    return time_arrival, ambulanceList, callList, timeline, next_event_time

In [46]:
def get_jobcount(timeline):
    timediff = np.append(np.diff(timeline['Timestamp']), 0)
    timeline['timediff'] = timediff
    
    n = timeline.shape[0]
    numCalls = np.zeros(n)
    
    count = 0
    for i in range(n):
        event = timeline.iloc[i]['Event']
        if event == 0:
            count += 1
        elif event == 5 or event == 7: 
            count -= 1
            
        if count <0:
            print("hi")
            
        numCalls[i] = count
        
    numCalls[-1] = numCalls[n-2]
    timeline['numCalls'] = numCalls
    return timeline

In [47]:
def get_jobs(timeline):
    total = max(timeline['Call'])
    
    arrival = np.zeros(total+1)*np.nan
    assign = np.zeros(total+1)*np.nan
    reach = np.zeros(total+1)*np.nan
    onsite = np.zeros(total+1)*np.nan
    transfer = np.zeros(total+1)*np.nan
    finish = np.zeros(total+1)*np.nan
    priority = np.zeros(total+1)
    
    for i in range(total + 1):
        c = timeline[timeline['Call'] == i]
        
        p = list(set(c['Priority']))[0]
        priority[i] = p
        
        n = c.shape[0]
        for index, row in c.iterrows():
            t = row['Timestamp']
            event = row['Event']
            if event == 0:
                arrival[i] = t
            elif event == 1:
                assign[i] = t if n > 1 else np.nan
            elif event == 2:
                reach[i] = t if n > 2 else np.nan
            elif event == 3:
                onsite[i] = t if n > 3 else np.nan
            elif event == 4:
                transfer[i] = t if n > 4 else np.nan
            elif event == 5:
                finish[i] = t if n > 5 else np.nan
            elif event == 7:
                finish[i] = t
#         print(n, arrival[i], assign[i], reach[i], onsite[i], transfer[i], finish[i])
        
    columns = ['priority', 'arrival_time', 'assigned_time', 'reach_patient', 'finish_onsite', 'reach_hospital', 'finish']
    data = list(zip(priority, arrival, assign, reach, onsite, transfer, finish))
    df = pd.DataFrame(data, columns=columns)
    
    df['waiting_time'] = df['assigned_time'] - df['arrival_time']
    df['total_time'] = df['finish'] - df['arrival_time']
    return df


In [48]:
def performance_metric(timeline, df, target, c = 4, verbose=True):
    
    result_dict = {}
    
    t = timeline.iloc[-1]['Timestamp']
    P = timeline.groupby('numCalls')['timediff'].sum() / t
    
    expectn = sum(P * P.index)
    try:
        expectw = sum(P[c+1:] * (P.index[c+1:]-c))
    except:
        expectw = sum(P[c+1:] * (P.index[c:]-c))
        
    utilization = (expectn - expectw) / c
    
    result_dict['totalCalls'] = df.shape[0]
    result_dict['utilization'] = utilization
    result_dict['expectNJobs'] = expectn
    result_dict['expectNQueue'] = expectw
    
    if verbose:
        print('Utilization:', utilization)
        print('Expected number of jobs in system:', expectn)
        print('Expected number of jobs in queue:', expectw)
    
    df_complete = df.dropna(axis=0)
    result_dict['expectedWaiting'] = np.mean(df_complete['waiting_time'])
    result_dict['expectedTotal'] = np.mean(df_complete['total_time'])
    result_dict['totalComplete'] = len(df_complete)
    
    if verbose:
        print('Expected time in queue:', np.mean(df_complete['waiting_time']))
        print('Expected time in system:', np.mean(df_complete['total_time']))
        print("Total completed patients: ",  len(df_complete))
    
    assigned = df[df['assigned_time'] > 0]
    count = 0
    for index, row in assigned.iterrows():
        if np.isnan(row['reach_patient']) or row['reach_patient']-row['arrival_time'] > target:
            count += 1
    
    result_dict['totalAssigned'] = len(assigned)
    result_dict['totalUnreachable'] = count
    result_dict['rateUnreachable'] = count / df.shape[0]
    
    reached = df[df['reach_patient'] > 0]
    result_dict['expectReach'] = np.mean(reached['reach_patient'] - reached['arrival_time'])
    
    if verbose:
        print("Total assigned patients: ", len(assigned))
        print("Total unreachable calls:", count)
        print("Portion of patients that is unreachable:", count/df.shape[0])
        print("Expected time to reach patients:", np.mean(reached['reach_patient'] - reached['arrival_time']))
    
    
    
    # Higher Priority
    highp = df[df['priority'] == 1]
    highp_complete = highp.dropna(axis=0)
    highp_assigned = highp[highp['assigned_time'] > 0]
    
    result_dict['totalHigh'] = len(highp)
    result_dict['totalCompleteHigh'] = len(highp_complete)
    result_dict['totalAssignedHigh'] = len(highp_assigned)
    
    if verbose:
        print("Total high priority patients: ",  len(highp))
        print("Total high priority patients completed: ",  len(highp_complete))
        print("Total high priority patients assigned: ",  len(highp_assigned))
    
    count = 0
    for index, row in highp_assigned.iterrows():
        if np.isnan(row['reach_patient']) or row['reach_patient']-row['arrival_time'] > target:
            count += 1
    
    highp_reached = highp[highp['reach_patient'] > 0]
    
    result_dict['expectReachHigh'] = np.mean(highp_reached['reach_patient'] - highp_reached['arrival_time'])
    result_dict['totalUnreachableHigh'] = count
    result_dict['expectWaitingHigh'] = np.mean(highp_complete['waiting_time'])
    result_dict['expectTotalHigh'] = np.mean(highp_complete['total_time'])
    result_dict['rateUnreachableHigh'] = count/len(highp)
    
    if verbose:
        print("Total high priority unreachable calls:", count)
        print("Portion of high priority calls that is unreachable:", count/len(highp))
        print('Expected time in queue (high priority patients):', np.mean(highp_complete['waiting_time']))
        print('Expected time in system (high priority patients):', np.mean(highp_complete['total_time']))
        print("Expected time to reach high priority patients:", result_dict['expectReachHigh'])
    
    # Lower Priority
    lowp = df[df['priority'] == 0]
    lowp_complete = lowp.dropna(axis=0)
    lowp_assigned = lowp[lowp['assigned_time'] > 0]

    result_dict['totalLow'] = len(lowp)
    result_dict['totalCompleteLow'] = len(lowp_complete)
    result_dict['totalAssignedLow'] = len(lowp_assigned)
    
    if verbose:
        print("Total low priority patients: ",  len(lowp))
        print("Total low priority patients completed: ",  len(lowp_complete))
        print("Total low priority patients assigned: ",  len(lowp_assigned))
    
    count = 0
    for index, row in lowp_assigned.iterrows():
        if np.isnan(row['reach_patient']) or row['reach_patient']-row['arrival_time'] > target:
            count += 1
            
    lowp_reached = lowp[lowp['reach_patient'] > 0]
    
    result_dict['expectReachLow'] = np.mean(lowp_reached['reach_patient'] - lowp_reached['arrival_time'])
    result_dict['totalUnreachableLow'] = count
    result_dict['expectWaitingLow'] = np.mean(lowp_complete['waiting_time'])
    result_dict['expectTotalLow'] = np.mean(lowp_complete['total_time'])
    result_dict['rateUnreachableLow'] = count/len(lowp)
    
    if verbose:
        print("Total low priority unreachable calls:", count)
        print("Portion of low priority calls that is unreachable:", count/len(lowp))
        print('Expected time in queue (low priority patients):', np.mean(lowp_complete['waiting_time']))
        print('Expected time in system (low priority patients):', np.mean(lowp_complete['total_time']))
        print("Expected time to reach high priority patients:", result_dict['expectReachLow'])
    
    
    return result_dict



# Set up the City

In [49]:
# Key parameters
city_dimension = 150 # assume square
num_hospital = 2
num_base = 6
time_threshold = 9 # delta
horizon = 60*24*7

M = 20
ambulance_speed = 30
call_mean = 4
call_priority = 0.25

# Heuristic Policies

## FCFS

In [24]:
city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i], speed = ambulance_speed) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Priority', 'Ambulance', 'Event', 'Timestamp'])

In [25]:
start = datetime.datetime.now()
time = 0

while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(get_first, time_arrival, ambulanceList, callList, timeline, city, M, time_threshold,
                  call_mean = call_mean, call_priority = call_priority)
    
end = datetime.datetime.now()
   
# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, 
# get_fcfs_threshold_else_nearest, get_nearest_threshold_else_fcfs

In [26]:
print((end - start).total_seconds())

48.186125


In [27]:
timeline = get_jobcount(timeline)
df = get_jobs(timeline)
fcfs_perform = performance_metric(timeline, df, time_threshold, c= num_base)

Utilization: 0.8061106265763609
Expected number of jobs in system: 6.337381630825941
Expected number of jobs in queue: 1.5007178713677765
Expected time in queue: 6.07328681992752
Expected time in system: 24.96183404164838
Total completed patients:  2557
Total assigned patients:  2562
Total unreachable calls: 821
Portion of patients that is unreachable: 0.3197040498442368
Expected time to reach patients: 8.461062566062704
Total high priority patients:  628
Total high priority patients completed:  626
Total high priority patients assigned:  627
Total high priority unreachable calls: 186
Portion of high priority calls that is unreachable: 0.2961783439490446
Expected time in queue (high priority patients): 5.495797249683027
Expected time in system (high priority patients): 24.55374945096034
Expected time to reach high priority patients: 7.859896928228217
Total low priority patients:  1940
Total low priority patients completed:  1931
Total low priority patients assigned:  1935
Total low pri

## FCFS with Priority

In [24]:
priority_vec = np.linspace(0.1,0.5, 5)

In [25]:
for pr in priority_vec:
    
    print("\nPriority Call Ratio: ", pr)
    
    city = City(city_dimension, city_dimension, num_hospital, num_base)
    ambulanceList = [Ambulance(city.base_location[i], speed = ambulance_speed) for i in range(len(city.base_location))]
    callList = {}
    time_arrival = 0
    timeline = pd.DataFrame(columns = ['Call', 'Priority', 'Ambulance', 'Event', 'Timestamp'])
    
    # start = datetime.datetime.now()
    time = 0
    while time < horizon:
        time_arrival, ambulanceList, callList, timeline, time = \
        get_next_event(get_first_highpriority, time_arrival, ambulanceList, callList, timeline, city, M, time_threshold,
                      call_mean = call_mean, call_priority = pr)


    # end = datetime.datetime.now()
    
    timeline = get_jobcount(timeline)
    df = get_jobs(timeline)
    fcfs_wPriority_perform = performance_metric(timeline, df, time_threshold, c= num_base)

# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, 
# get_fcfs_threshold_else_nearest, get_nearest_threshold_else_fcfs
    
    


Priority Call Ratio:  0.1
Utilization: 0.7917436930221333
Expected number of jobs in system: 5.817581687920397
Expected number of jobs in queue: 1.0671195297875966
Expected time in queue: 4.514293819282507
Expected time in system: 23.531438530177848
Total completed patients:  2492
Total assigned patients:  2492
Total unreachable calls: 626
Portion of patients that is unreachable: 0.251103088648215
Expected time to reach patients: 6.950367947013297
Total high priority patients:  251
Total high priority patients completed:  251
Total high priority patients assigned:  251
Total high priority unreachable calls: 11
Portion of high priority calls that is unreachable: 0.043824701195219126
Expected time in queue (high priority patients): 1.536058705752513
Expected time in system (high priority patients): 20.86213186104013
Expected time to reach high priority patients: 3.990022811045725
Total low priority patients:  2242
Total low priority patients completed:  2241
Total low priority patients 

## Neareset

In [30]:
city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i], speed = ambulance_speed) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Priority', 'Ambulance', 'Event', 'Timestamp'])

In [31]:
start = datetime.datetime.now()
time = 0
while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(get_nearest, time_arrival, ambulanceList, callList, timeline, city, M, time_threshold,
                   call_mean = call_mean, call_priority = call_priority)


end = datetime.datetime.now()

# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, 
# get_fcfs_threshold_else_nearest, get_nearest_threshold_else_fcfs

In [32]:
timeline = get_jobcount(timeline)
df = get_jobs(timeline)
nearest_perform = performance_metric(timeline, df, time_threshold, c= num_base)

Utilization: 0.7778588281245726
Expected number of jobs in system: 5.868675756475721
Expected number of jobs in queue: 1.2015227877282852
Expected time in queue: 4.757908842239621
Expected time in system: 23.4560472601962
Total completed patients:  2492
Total assigned patients:  2497
Total unreachable calls: 439
Portion of patients that is unreachable: 0.17531948881789136
Expected time to reach patients: 7.2437853144418805
Total high priority patients:  635
Total high priority patients completed:  632
Total high priority patients assigned:  633
Total high priority unreachable calls: 124
Portion of high priority calls that is unreachable: 0.1952755905511811
Expected time in queue (high priority patients): 5.630302871990372
Expected time in system (high priority patients): 23.99677184539122
Expected time to reach high priority patients: 8.17092453261407
Total low priority patients:  1869
Total low priority patients completed:  1860
Total low priority patients assigned:  1864
Total low pr

## Nearest with High Priority

In [33]:
city = City(city_dimension, city_dimension, num_hospital, num_base)
ambulanceList = [Ambulance(city.base_location[i], speed = ambulance_speed) for i in range(len(city.base_location))]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Priority', 'Ambulance', 'Event', 'Timestamp'])

In [34]:
start = datetime.datetime.now()
time = 0
while time < horizon:
    time_arrival, ambulanceList, callList, timeline, time = \
    get_next_event(get_nearest_highpriority, time_arrival, ambulanceList, callList, timeline, city, M, time_threshold,
                  call_mean = call_mean, call_priority = call_priority)


end = datetime.datetime.now()

# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, 
# get_fcfs_threshold_else_nearest, get_nearest_threshold_else_fcfs

In [35]:
timeline = get_jobcount(timeline)
df = get_jobs(timeline)
nearest_wPriority_perform = performance_metric(timeline, df, time_threshold, c= num_base)

Utilization: 0.8072802766820325
Expected number of jobs in system: 5.9823776926270416
Expected number of jobs in queue: 1.1386960325348463
Expected time in queue: 4.672862049895625
Expected time in system: 23.80169843772837
Total completed patients:  2532
Total assigned patients:  2535
Total unreachable calls: 467
Portion of patients that is unreachable: 0.18414826498422712
Expected time to reach patients: 7.151116592802982
Total high priority patients:  647
Total high priority patients completed:  646
Total high priority patients assigned:  646
Total high priority unreachable calls: 43
Portion of high priority calls that is unreachable: 0.06646058732612056
Expected time in queue (high priority patients): 1.544980379304677
Expected time in system (high priority patients): 20.468896738380824
Expected time to reach high priority patients: 4.01184827316524
Total low priority patients:  1889
Total low priority patients completed:  1886
Total low priority patients assigned:  1889
Total low 

## FCFS within Threshold, then Nearest

In [50]:
distance_threshold_vec = np.arange(1,5)
horizon = 60*24*4

In [51]:
for dt in distance_threshold_vec:
    print("\nDistance Threshold Ratio: ", dt)
    
    city = City(city_dimension, city_dimension, num_hospital, num_base)
    ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
    callList = {}
    time_arrival = 0
    timeline = pd.DataFrame(columns = ['Call', 'Priority', 'Ambulance', 'Event', 'Timestamp'])
    
    # start = datetime.datetime.now()
    time = 0
    while time < horizon:
        time_arrival, ambulanceList, callList, timeline, time = \
        get_next_event(get_fcfs_threshold_else_nearest, time_arrival, ambulanceList, callList, timeline, 
                       city, M, time_threshold, 
                       dt, call_mean = call_mean, call_priority = call_priority)


    # end = datetime.datetime.now()
    
    timeline = get_jobcount(timeline)
    df = get_jobs(timeline)
    thresh_fcfs_perform = performance_metric(timeline, df, time_threshold, c= num_base)

# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, 
# get_fcfs_threshold_else_nearest, get_nearest_threshold_else_fcfs


Distance Threshold Ratio:  1
Utilization: 0.8081057956544813
Expected number of jobs in system: 6.437099109747128
Expected number of jobs in queue: 1.5884643358202404
Expected time in queue: 6.5463452396670165
Expected time in system: 25.71067353285162
Total completed patients:  1440
Total assigned patients:  1442
Total unreachable calls: 446
Portion of patients that is unreachable: 0.3086505190311419
Expected time to reach patients: 8.98874558380998
Total high priority patients:  339
Total high priority patients completed:  338
Total high priority patients assigned:  339
Total high priority unreachable calls: 94
Portion of high priority calls that is unreachable: 0.27728613569321536
Expected time in queue (high priority patients): 6.543790783821032
Expected time in system (high priority patients): 25.43897733897142
Expected time to reach high priority patients: 8.953098480438326
Total low priority patients:  1106
Total low priority patients completed:  1102
Total low priority patient

## Threshold supplemented with FCFS

In [37]:
for dt in distance_threshold_vec:
    print("\nDistance Threshold Ratio: ", dt)
    
    city = City(city_dimension, city_dimension, num_hospital, num_base)
    ambulanceList = [Ambulance(city.base_location[i]) for i in range(len(city.base_location))]
    callList = {}
    time_arrival = 0
    timeline = pd.DataFrame(columns = ['Call', 'Priority', 'Ambulance', 'Event', 'Timestamp'])
    
    # start = datetime.datetime.now()
    time = 0
    while time < horizon:
        time_arrival, ambulanceList, callList, timeline, time = \
        get_next_event(get_nearest_threshold_else_fcfs, time_arrival, ambulanceList, callList, timeline, 
                       city, M, time_threshold, 
                       dt, call_mean = call_mean, call_priority = call_priority)


    # end = datetime.datetime.now()
    
    timeline = get_jobcount(timeline)
    df = get_jobs(timeline)
    thresh_fcfs_perform = performance_metric(timeline, df, time_threshold, c= num_base)

# policies are:
# get_first, get_first_highpriority, 
# get_nearest, get_nearest_highpriority, 
# get_fcfs_threshold_else_nearest, get_nearest_threshold_else_fcfs


Distance Threshold Ratio:  1
Utilization: 0.737497880549811
Expected number of jobs in system: 5.13770798277616
Expected number of jobs in queue: 0.7127206994772941
Expected time in queue: 3.0799972183282063
Expected time in system: 20.97365122096065
Total completed patients:  2460
Total assigned patients:  2465
Total unreachable calls: 298
Portion of patients that is unreachable: 0.12079448723145521
Expected time to reach patients: 5.45486169500012
Total high priority patients:  625
Total high priority patients completed:  623
Total high priority patients assigned:  624
Total high priority unreachable calls: 78
Portion of high priority calls that is unreachable: 0.1248
Expected time in queue (high priority patients): 3.665389940623071
Expected time in system (high priority patients): 22.051658160661184
Expected time to reach high priority patients: 6.0925018420870805
Total low priority patients:  1842
Total low priority patients completed:  1837
Total low priority patients assigned: 

In [38]:
def print_ambulance(ambulanceList):
    statusList = []
    originList = []
    destinationList = []
    timeList = []
    endtimeList = []
    
    for ambulance in ambulanceList:
        statusList.append(ambulance.status)
        originList.append(ambulance.origin)
        destinationList.append(ambulance.destination)
        timeList.append(ambulance.time)
        endtimeList.append(ambulance.endtime)
    
    return statusList, originList, destinationList, timeList, endtimeList
    

In [39]:
def print_call(callList):
    
    statusList = []
    locationList = []
    timeList = []
    
    for call_index in callList:
        call = callList[call_index]
        statusList.append(call.status)
        locationList.append(call.location)
        timeList.append(call.arrival_time)
    
    return statusList, locationList, timeList

In [40]:
def get_unreachable_calls(time, ambulanceList, callList, delta, speed = 10):
    
    unreach_calls = 0
    
    for ambulance in ambulanceList:
        if ambulance.status == 1:
            target_call = callList[ambulance.call]
            target_threshold = target_call.arrival_time + delta
            reach_outside = ambulance.endtime > target_threshold
            unreach_calls +=1
    
    return unreach_calls
            

In [41]:
def get_ambulance_current_loc(ambulance, time):
    
    ambulance_origin = ambulance.origin
    ambulance_destination = ambulance.destination
    
    ambulance_start = ambulance.time
    ambulance_end = ambulance.endtime
    
    current_time = time
    travel_ratio = (current_time - ambulance_start) / (ambulance_end - ambulance_start)
    
    current_location = ((ambulance_destination[0] - ambulance_origin[0]) * travel_ratio + ambulance_origin[0],
                        (ambulance_destination[1] - ambulance_origin[1]) * travel_ratio + ambulance_origin[1])
    
    return current_location

In [42]:
def get_uncovered_calls(time, ambulanceList, callList, delta, city, speed = 10):
    
    uncovered_base = []
    for base in city.base_location:
        cover_calls = 0
        for ambulance in ambulanceList:
            if ambulance.status == 0 or ambulance.status == 5:
                current_location = get_ambulance_current_loc(ambulance, time)
                distance_to_base = get_distance(current_location, base)
                time_to_base = get_ambulance_travel_time(distance_to_base, speed = speed)
                
                if time_to_base < delta:
                    cover_calls += 1
                
        
        if cover_calls == 0:
            uncovered_base.append(1)
        else:
            uncovered_base.append(0)
            
    return uncovered_base
            